# SVM-Lin

In [1]:
#load the necessary packages:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using Theano backend.


All packages successfully loaded


In [2]:
# load the necessary file:
file = 'MTBLS136.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 1649 TOTAL PEAKS: 949
Done!


In [3]:
# Clean PeakTable and Extract PeakList
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(PercMiss < 20)] 
PeakList = PeakTableClean['Name']  

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=85)

# Extract Train Data 
XTrain = DataTrain[PeakList]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
XTest = DataTest[PeakList]                                     
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma) 
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)     

In [4]:
# Parameter Dictionary
C_range = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,  0.1]
param_dict = dict(C=C_range, kernel="linear")               

# Initialise
cv = cb.cross_val.KFold(model=cb.model.SVM,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)                            

# Run and Plot
cv.run()  
cv.plot(metric='auc')
cv.plot(metric='r2q2')

Number of cores set to: 128
Running ...


2/2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 67587.77it/s]


Time taken: 7.22 minutes with 128 cores
Done!


Loading BokehJS ...

Loading BokehJS ...

In [5]:
# Parameter Dictionary
#C_range = [1e-6,1e-5,1e-4,5e-4,1e-3,5e-3,1e-2,1e-1]
C_range = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001]
param_dict = dict(C=C_range, kernel="linear")               

# Initialise
cv = cb.cross_val.KFold(model=cb.model.SVM,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)                                 

# Run and Plot
cv.run()  
cv.plot(metric='auc')
cv.plot(metric='r2q2') 

Number of cores set to: 128
Running ...


2/2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 76163.14it/s]


Time taken: 0.85 minutes with 128 cores
Done!


Loading BokehJS ...

Loading BokehJS ...

In [6]:
# Build Model
model = cb.model.SVM(C=0.0005, kernel="linear")     
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]

# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest) 

Loading BokehJS ...

In [7]:
# Extract X Data
XBoot = DataTable2[PeakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100) 
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(trainset=EvalTrain, testset=EvalTest)  

Number of cores set to: 128


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 82744.21it/s]


Time taken: 0.17 minutes with 128 cores


Loading BokehJS ...

Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Theano backend.
Using Thea

In [12]:
important_feat = abs(model.model.coef_[0]) # extract coefficient
#get indices of those important features
idx = important_feat.argsort(kind= "quicksort")
idx= idx[::-1][:5]
#get more information from peaktable:
top_met = PeakList.iloc[idx]
top_met_info = PeakTable.iloc[top_met.index-1]
#5 most important metabolites
top_met_info

,Idx,Name,Label,Perc_missing
41,41,M41,1-linoleoyl-GPA (18:2)*,0.303214
618,618,M618,lysine,0.000000
123,123,M123,2-aminoheptanoate,0.000000
212,212,M212,4-acetamidobutanoate,0.121286
113,113,M113,2'-deoxyuridine,0.606428


# PCR

In [13]:
# load the necessary file:
file = 'MTBLS136.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 1649 TOTAL PEAKS: 949
Done!


In [14]:
# Clean PeakTable and Extract PeakList
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(PercMiss < 20)] 
PeakList = PeakTableClean['Name']  

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=85)

# Extract Train Data 
XTrain = DataTrain[PeakList]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
XTest = DataTest[PeakList]                                     
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma) 
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)   

In [15]:
# Parameter Dictionary
#param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]}  
param_dict = {'n_components': [20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,200]}

# Initialise
cv = cb.cross_val.KFold(model=cb.model.PCR,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)                                

# Run and Plot
cv.run()  
cv.plot(metric='r2q2')

Number of cores set to: 128
Running ...


2/2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [00:00<00:00, 23986.49it/s]


Time taken: 6.81 minutes with 128 cores
Done!


Loading BokehJS ...

In [16]:
# Build Model
model = cb.model.PCR(n_components=80)
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]

# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest) 

Loading BokehJS ...

In [17]:
# Extract X Data
XBoot = DataTable2[PeakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100) 
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(trainset=EvalTrain, testset=EvalTest)  

Number of cores set to: 128


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 52109.63it/s]


Time taken: 0.16 minutes with 128 cores


Loading BokehJS ...

In [18]:
important_feat = abs(model.model.coef_)
#get indices of those important features
idx = important_feat.argsort(kind= "quicksort")
idx= idx[::-1][:5]
#get more information from peaktable:
top_met = PeakList.iloc[idx]
top_met_info = PeakTable.iloc[top_met.index-1]
#5 most important metabolites
top_met_info

,Idx,Name,Label,Perc_missing
402,402,M402,cystine,0.060643
577,577,M577,isobutyrylcarnitine (C4),0.000000
886,886,M886,succinimide,5.275925
495,495,M495,gluconate,0.545785
593,593,M593,lactosyl-N-behenoyl-sphingosine (d18:1/22:0)*,2.728927


# RF

In [19]:
# load the necessary file:
file = 'MTBLS136.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 1649 TOTAL PEAKS: 949
Done!


In [20]:
# Clean PeakTable and Extract PeakList
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(PercMiss < 20)] 
PeakList = PeakTableClean['Name']  

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=85)

# Extract Train Data 
XTrain = DataTrain[PeakList]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
XTest = DataTest[PeakList]                                     
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma) 
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)         

In [21]:
# Parameter Dictionary
depth = list(range(1,11))
leaf_asfraction = [0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]

param_dict = dict(max_depth=depth,
                  min_samples_leaf=leaf_asfraction,
                  max_features='sqrt',
                  criterion='gini',
                  min_samples_split=2,
                  max_leaf_nodes=None,
                  n_estimators=100)

# Initialise
cv = cb.cross_val.KFold(model=cb.model.RF,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)       
                              

# Run and Plot
cv.run()  
cv.plot(metric='auc', color_beta=[5,5,3])
cv.plot(metric='r2q2', color_beta=[5,5,3])

Number of cores set to: 128
Running ...


2/2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1100/1100 [00:42<00:00, 25.87it/s]


Time taken: 1.04 minutes with 128 cores
Done!


/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a

Loading BokehJS ...

/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/home/anepal/.conda/envs/process_env/envs/MetabComparisonBinaryML/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a

Loading BokehJS ...

In [22]:
# Build Model
model = cb.model.RF(max_depth=7,
                    min_samples_leaf=0.05,
                    max_features='sqrt',
                    criterion='gini',
                    min_samples_split=2,
                    max_leaf_nodes=None,
                    n_estimators=100) 
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]

# Evaluate Model (include Test Dataset
model.evaluate(testset=EvalTest) 

Loading BokehJS ...

In [23]:
# Extract X Data
XBoot = DataTable2[PeakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100) 
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(trainset=EvalTrain, testset=EvalTest)   

Number of cores set to: 128


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 77888.65it/s]


Time taken: 0.21 minutes with 128 cores


Loading BokehJS ...

In [24]:
# for important metabolies / features:
important_feat = abs(model.model.feature_importances_)
#get indices of those important features
idx = important_feat.argsort(kind= "quicksort")
idx= idx[::-1][:5]
#get more information from peaktable:
top_met = PeakList.iloc[idx]
top_met_info = PeakTable.iloc[top_met.index-1]
#5 most important metabolites
top_met_info

,Idx,Name,Label,Perc_missing
84,84,M84,1-stearoyl-2-arachidonoyl-GPC (18:0/20:4),0.000000
41,41,M41,1-linoleoyl-GPA (18:2)*,0.303214
39,39,M39,1-linoleoyl-2-arachidonoyl-GPC (18:2/20:4n6)*,1.030928
38,38,M38,1-linolenoylglycerol (18:3),0.242571
143,143,M143,2-linoleoylglycerol (18:2),0.545785
